<a href="https://colab.research.google.com/github/pollinations/hive/blob/main/interesting_notebooks/LAFITE_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
!git clone https://github.com/drboog/Lafite
%cd /content/Lafite
!pip install git+https://github.com/openai/CLIP.git
!pip install ninja

/content
fatal: destination path 'Lafite' already exists and is not an empty directory.
/content/Lafite
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-slhkye99
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-slhkye99
     |████████████████████████████████| 108 kB 10.3 MB/s 


In [ ]:
!pip install torch==1.9.1 torchtext==0.10.1 torchvision==0.10.1 torchaudio==0.10.1
 

  Using cached torch-1.9.1-cp37-cp37m-manylinux1_x86_64.whl (831.4 MB)
  Using cached torchtext-0.10.1-cp37-cp37m-manylinux1_x86_64.whl (7.6 MB)
  Using cached torchvision-0.10.1-cp37-cp37m-manylinux1_x86_64.whl (22.1 MB)
  Using cached torchaudio-0.10.1-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchtext to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install torch==1.9.1, torchaudio==0.10.1, torchtext==0.10.1 and torchvision==0.10.1 bec

In [ ]:
from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import torch
import numpy as np
import pickle
import os
import clip
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import dnnlib, legacy
import clip
import torch.nn.functional as F
import torchvision.transforms as T
from tqdm import tqdm
import scipy

ModuleNotFoundError: ignored

In [ ]:
class Generator:
    def __init__(self, device, path):
        self.name = 'generator'
        self.model = self.load_model(device, path)
        self.device = device
        self.force_32 = False
        
    def load_model(self, device, path):
        with dnnlib.util.open_url(path) as f:
            network= legacy.load_network_pkl(f)
            self.G_ema = network['G_ema'].to(device)
            self.D = network['D'].to(device)
#                 self.G = network['G'].to(device)
            return self.G_ema
        
    def generate(self, z, c, fts, noise_mode='const', return_styles=True):
        return self.model(z, c, fts=fts, noise_mode=noise_mode, return_styles=return_styles, force_fp32=self.force_32)
    
    def generate_from_style(self, style, noise_mode='const'):
        ws = torch.randn(1, self.model.num_ws, 512)
        return self.model.synthesis(ws, fts=None, styles=style, noise_mode=noise_mode, force_fp32=self.force_32)
    
    def tensor_to_img(self, tensor):
        img = torch.clamp((tensor + 1.) * 127.5, 0., 255.)
        img_list = img.permute(0, 2, 3, 1)
        img_list = [img for img in img_list]
        return Image.fromarray(torch.cat(img_list, dim=-2).detach().cpu().numpy().astype(np.uint8))

In [7]:
with torch.no_grad():

    device = 'cuda:0' # please use GPU, do not use CPU
    path = '/content/COCO2014_Language-free_Gaussian.pkl'  # pre-trained model
    generator = Generator(device=device, path=path)
    clip_model, _ = clip.load("ViT-B/32", device=device)
    clip_model = clip_model.eval()
    
    txt = 'an armchair in the shape of an avocado'  # input sentence
    tokenized_text = clip.tokenize([txt]).to(device)
    txt_fts = clip_model.encode_text(tokenized_text)
    txt_fts = txt_fts/txt_fts.norm(dim=-1, keepdim=True)
    
    z = torch.randn((1, 512)).to(device)
    c = torch.randn((1, 1)).to(device) # label is actually not used
    img, _ = generator.generate(z=z, c=c, fts=txt_fts)
    to_show_img = generator.tensor_to_img(img)
    to_show_img.save('./generated.jpg')
